In [1]:
from transformers import BertModel, BertConfig, BertTokenizer, AdamW
import torch
import torch.nn as nn


c:\Users\pc\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import hazm
import string
from preprocessing import Preprocessing
import pandas as pd
from tqdm import tqdm

train_set = pd.read_csv('datasets/arman-text-emotion-main/train.csv')
val_set = pd.read_csv('datasets/arman-text-emotion-main/test.csv')

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModel

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
tokenizer.encode_plus('سلام بر تو ای')

{'input_ids': [2, 3132, 2043, 2092, 2156, 4], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [5]:
tokenizer.save_pretrained("fintuned_models/tokenizer")
model.save_pretrained("fintuned_models/AutoModelParsBert")

In [6]:
for param in model.parameters():
    param.requires_grad = False

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# class ParsBERT(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.bert = model
#         for param in self.bert.parameters():
#             param.requires_grad = False

#         self.fc = nn.Linear(768,7)

#     def forward(self, input_ids, attention_mask,token_type_ids):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
#         last_hidden_state = F.relu(outputs.last_hidden_state)
#         pooled_output = torch.mean(last_hidden_state, dim=1)
#         logits = self.fc(pooled_output)
#         return logits
    
class ParsBERT(nn.Module):
    def __init__(self):
        super(ParsBERT, self).__init__()
        self.model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
        for param in self.model.parameters():
            param.requires_grad = False

        self.fc = nn.Linear(self.model.config.hidden_size, 7)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = F.relu(outputs.last_hidden_state)
        pooled_output = torch.mean(last_hidden_state, dim=1)
        logits = self.fc(pooled_output)
        return logits

    
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

In [8]:
import pandas as pd

preprocessor = Preprocessing('arman')

class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.texts = data['text'].apply(preprocessor).tolist()
        self.labels = data['label'].tolist()
        self.num_classes = len(set(self.labels))
        self.labels_dict = {'SAD': 0, 'HAPPY': 1, 'OTHER': 2, 'SURPRISE': 3, 'FEAR': 4, 'HATE': 5, 'ANGRY': 6}
        self.tokenizer = tokenizer
        self.max_len = 64

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels_dict[self.labels[idx]]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        inputs = {
            'text': text,
            'target':label,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }
        return inputs
    

In [9]:
from sklearn.metrics import f1_score
from tqdm import tqdm
import config

def train(model, train_dataset, valid_dataset, num_epochs, batch_size, criterion, optimizer, device):
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)
    total_steps = len(train_loader) * num_epochs
    model = model.to(device)
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()
        epoch_loss = 0
        epoch_correct = 0
        epoch_total = 0
        epoch_f1_score = 0

        for batch in tqdm(train_loader):
            inputs = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['target'].to(device)

            optimizer.zero_grad()
            outputs = model(inputs, attention_mask, token_type_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, dim=1)
            epoch_correct += (predicted == labels).sum().item()
            epoch_total += labels.size(0)
            epoch_f1_score += f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')

        epoch_loss /= len(train_loader)
        epoch_accuracy = epoch_correct / epoch_total
        epoch_f1_score /= len(train_loader)
        print(f"Train Loss: {epoch_loss:.4f} | Train Accuracy: {epoch_accuracy:.4f} | Train F1 Score: {epoch_f1_score:.4f}")

        # Evaluate on the validation set
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        val_f1_score = 0

        with torch.no_grad():
            for batch in tqdm(valid_loader):
                inputs = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                labels = batch['target'].to(device)

                outputs = model(inputs, attention_mask, token_type_ids)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, dim=1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
                val_f1_score += f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')

        val_loss /= len(valid_loader)
        val_accuracy = val_correct / val_total
        val_f1_score /= len(valid_loader)
        print(f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f} | Val F1 Score: {val_f1_score:.4f}")
        with open(config.LOGFILE, 'a') as f:
                f.write(f"Epoch {epoch+1}/{num_epochs} | ")
                f.write(f"Train Loss: {epoch_loss:.4f} | ")
                f.write(f"Train Accuracy: {epoch_accuracy:.4f} | ")
                f.write(f"Train F1 Score: {epoch_f1_score:.4f} | ")
                f.write(f"Valid Loss: {val_loss:.4f} | ")
                f.write(f"Valid Accuracy: {val_accuracy:.4f} | ")
                f.write(f"Valid F1 Score: {val_f1_score:.4f} | ")
                f.write("\n")
                
        checkpoint = {
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_loss': epoch_loss,
                    'train_acc': epoch_accuracy,
                    'train_f1_score': epoch_f1_score,
                    'valid_loss': val_loss,
                    'valid_acc': val_accuracy,
                    'valid_f1_score': val_f1_score
                }
        torch.save(checkpoint, 'checkpoints/model_checkpoint.pth')

# Example usage
# Replace with your model
model = ParsBERT()
train_dataset = EmotionDataset(train_set)  # Replace with your dataset
val_dataset = EmotionDataset(val_set)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
train(model,train_dataset,val_dataset, num_epochs=4000, batch_size=64, criterion=criterion, optimizer=optimizer, device=device)


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/4000


100%|██████████| 97/97 [00:41<00:00,  2.36it/s]


Train Loss: 1.7334 | Train Accuracy: 0.3174 | Train F1 Score: 0.1789


100%|██████████| 18/18 [00:07<00:00,  2.36it/s]


...

Train Loss: 1.1114 | Train Accuracy: 0.5912 | Train F1 Score: 0.5802


100%|██████████| 18/18 [00:06<00:00,  2.60it/s]


Val Loss: 1.5303 | Val Accuracy: 0.4457 | Val F1 Score: 0.3852
Epoch 58/4000


 48%|████▊     | 47/97 [00:20<00:21,  2.34it/s]


KeyboardInterrupt: 

In [4]:
import utils
import torch
import pandas as pd


df = pd.read_csv('datasets/1.intro_data_text.csv',encoding='utf-8')
# Read the CSV file

# Select relevant columns and rename them
df = df[['tweet', 'primary_emotion']]
df.columns = ['text', 'label']

# Save the modified dataset
df.to_csv('modified_dataset.csv', index=False)